In [21]:
#Imported our dependencies 
#downloaded alot of dependencies 
import os
import json
import pickle
import os.path
import requests
import pandas as pd
from scipy import stats
from config import api_key
import googleapiclient.errors
import matplotlib.pyplot as plt
import googleapiclient.discovery
from __future__ import print_function
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pprint

In [22]:
#Imported and using build() function to build the query url to get our specific data
#used search_cache.json to store a copy of the data on to our system to we dont run out of youtube api quota of the day 
#made it into a dataframe
youtube = build("youtube", "v3", developerKey=api_key)
#snippet_features = ["categoryId", "publishedAt", "channelId", "channelTitle"]
api_run = True 
if not api_run:
    most_viewed = youtube.search().list(part=["snippet", "id"],
                                           type="video", 
                                           publishedAfter="2020-03-15T00:00:00Z", 
                                           publishedBefore="2020-07-15T00:00:00Z",
                                           videoDuration="any", 
                                           order="viewCount", 
                                           maxResults=50)
    most_viewed = most_viewed.execute()
    #print(json.dumps(youtube_top50["items"][0],indent=4))
    df_mostviewed = pd.DataFrame(most_viewed['items'])
    df_mostviewed.to_json('SEARCH_CACHE.json') 
df_mostviewed=pd.DataFrame(pd.read_json('SEARCH_CACHE.json'))      
df_mostviewed.head()

,kind,etag,id,snippet
0,youtube#searchResult,uXbPHe9bjIs1lZSAAAorF0nvu9M,"{'kind': 'youtube#video', 'videoId': 'iL53Y28R...","{'publishedAt': '2020-03-15T01:45:33Z', 'chann..."
1,youtube#searchResult,2KrYMR0fJMrD8RBYyqD0UkKZQUA,"{'kind': 'youtube#video', 'videoId': 'NMre6IAA...","{'publishedAt': '2020-04-17T12:57:56Z', 'chann..."
2,youtube#searchResult,2IBIJkiyBrDzVyc6k3YQgh7TCI0,"{'kind': 'youtube#video', 'videoId': 'tkUvWJiT...","{'publishedAt': '2020-03-19T02:42:21Z', 'chann..."
3,youtube#searchResult,qrdXUiBNcr_CZPubgCX3GZr1kLY,"{'kind': 'youtube#video', 'videoId': 'd1iwUB9Y...","{'publishedAt': '2020-03-24T15:43:55Z', 'chann..."
4,youtube#searchResult,hEbLZ43Sm1m1AebLF3buS1YKShs,"{'kind': 'youtube#video', 'videoId': '0k25aI62...","{'publishedAt': '2020-04-15T10:13:41Z', 'chann..."


In [23]:
#normalized the data so we can read/use it easily for analysis
#merged the dataframes together 
df_video_ids = pd.json_normalize(df_top50['id'])
df_snippets = pd.json_normalize(df_top50['snippet']) 
df_merge = pd.merge(df_video_ids , df_snippets, left_index=True, right_index=True)
df_merge.head(1)

,kind,videoId,publishedAt,channelId,title,description,channelTitle,liveBroadcastContent,publishTime,thumbnails.default.url,thumbnails.default.width,thumbnails.default.height,thumbnails.medium.url,thumbnails.medium.width,thumbnails.medium.height,thumbnails.high.url,thumbnails.high.width,thumbnails.high.height
0,youtube#video,iL53Y28Rp84,2020-03-15T01:45:33Z,UCf8w5m0YsRa8MHQ5bwSGmbw,Asianet News Live TV | Malayalam News Live | ഏ...,Asianet News Youtube Live delivers breaking an...,asianetnews,live,2020-03-15T01:45:33Z,https://i.ytimg.com/vi/iL53Y28Rp84/default_liv...,120,90,https://i.ytimg.com/vi/iL53Y28Rp84/mqdefault_l...,320,180,https://i.ytimg.com/vi/iL53Y28Rp84/hqdefault_l...,480,360


In [24]:
#request another api search by youtube.video().list for the statistics information on the top videos 
#made that into a dataframe
request = youtube.videos().list(part='id, statistics, snippet', id=list(df_top50_merge['videoId']))
response = request.execute()# json
df_stat = pd.DataFrame(response['items']) # response['items'] list of dicts
df_stat.head()

,kind,etag,id,snippet,statistics
0,youtube#video,TfNgRL-AjZyOzc9UO1gaUiUw7Hg,iL53Y28Rp84,"{'publishedAt': '2020-03-15T01:45:33Z', 'chann...","{'viewCount': '319752506', 'likeCount': '29660..."
1,youtube#video,IP1fGBowwGvFsrN_x6yRZnSDYoI,NMre6IAAAiU,"{'publishedAt': '2020-04-17T12:57:56Z', 'chann...","{'viewCount': '182365455', 'likeCount': '27416..."
2,youtube#video,uim7yD3oFOaLRJTU32imGRWVD68,tkUvWJiTf9A,"{'publishedAt': '2020-03-19T02:42:21Z', 'chann...","{'viewCount': '133849157', 'likeCount': '17930..."
3,youtube#video,AN_6IoVHrh2P3BllBcBbpO2KN8o,d1iwUB9YFnA,"{'publishedAt': '2020-03-24T15:43:55Z', 'chann...","{'viewCount': '114980101', 'likeCount': '12470..."
4,youtube#video,2_XIj0czqYjCNRaWD3WpQ6kJCk4,0k25aI62ENU,"{'publishedAt': '2020-04-15T10:13:41Z', 'chann...","{'viewCount': '83959014', 'likeCount': '22', '..."


In [28]:
#organized the data that was nested in the video id by json.normalize
df_cleanstat = pd.json_normalize(df_stat['statistics']) 
df_cleanstat.head()

,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
0,319752506,296608,107384,0,0
1,182365455,274168,28158,0,0
2,133849157,179301,33630,0,0
3,114980101,124701,25054,0,0
4,83959014,22,7,0,NaN


In [26]:
#merging the original data and the stats 
df_stat_merge = pd.merge(df_merge, df_cleanstat, left_index=True, right_index=True)
df_stat_merge.head(1)

,kind,videoId,publishedAt,channelId,title,description,channelTitle,liveBroadcastContent,publishTime,thumbnails.default.url,...,thumbnails.medium.width,thumbnails.medium.height,thumbnails.high.url,thumbnails.high.width,thumbnails.high.height,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
0,youtube#video,iL53Y28Rp84,2020-03-15T01:45:33Z,UCf8w5m0YsRa8MHQ5bwSGmbw,Asianet News Live TV | Malayalam News Live | ഏ...,Asianet News Youtube Live delivers breaking an...,asianetnews,live,2020-03-15T01:45:33Z,https://i.ytimg.com/vi/iL53Y28Rp84/default_liv...,...,320,180,https://i.ytimg.com/vi/iL53Y28Rp84/hqdefault_l...,480,360,319752506,296608,107384,0,0


In [29]:
#cleaned up the final dataframe, got rid of the columns that was irrelevant to the project
df_combined = df_stat_merge.drop(columns=['kind','liveBroadcastContent', 'publishTime', 'thumbnails.default.url', 'thumbnails.medium.width', 'thumbnails.default.width', 'thumbnails.high.width', 'thumbnails.high.height', 'thumbnails.default.height', 'thumbnails.medium.url', 'thumbnails.medium.height', 'thumbnails.high.url', 'favoriteCount', 'commentCount'])
df_combined

,videoId,publishedAt,channelId,title,description,channelTitle,viewCount,likeCount,dislikeCount
0,iL53Y28Rp84,2020-03-15T01:45:33Z,UCf8w5m0YsRa8MHQ5bwSGmbw,Asianet News Live TV | Malayalam News Live | ഏ...,Asianet News Youtube Live delivers breaking an...,asianetnews,319752506,296608,107384
1,NMre6IAAAiU,2020-04-17T12:57:56Z,UCDGiCfCZIV5phsoGiPwIcyQ,[LIVE] Coronavirus Pandemic: Real Time Counter...,"Coronavirus Live Streaming: Breaking news, wor...",Roylab Stats,182365455,274168,28158
2,tkUvWJiTf9A,2020-03-19T02:42:21Z,UCeJWZgSMlzqYEDytDnvzHnw,"LIVE Updates | Top Regional, National and Inte...","LIVE Updates | Top Regional, National and Inte...",TV9 Gujarati,133849157,179301,33630
3,d1iwUB9YFnA,2020-03-24T15:43:55Z,UC-f7r46JhYv78q5pGrO6ivA,Mediaone News | Malayalam News Live | Malayal...,"malayalam news live, malayalam news, malayalam...",MediaoneTV Live,114980101,124701,25054
4,0k25aI62ENU,2020-04-15T10:13:41Z,UCszP2pSj7dKdFWmWMFS05jQ,Ensure®&#39;s #11ImmunityNutrients Helps You t...,Your immune system runs on the fuel you put in...,Ensure India,83959014,22,7
5,taAYP6uAGvs,2020-07-12T12:07:35Z,UCfiwzLy-8yKzIbsmZTzxDgw,Al Jazeera Arabic Live Stream HD- البث الحي لق...,للاشتراك في قناة الجزيرة على اليوتيوب: http://...,Al Jazeera Channel قناة الجزيرة,78630805,92127,22165
6,ThbXM1-Wfyw,2020-04-11T08:49:05Z,UCHPbrkmgqJpoKaGxrUmbMvA,#ОставайсяДома - Сериалы Сваты и Байки Митяя -...,Все сезоны и все серии популярных сериалов сту...,Фильмы и Сериалы Студии Квартал 95,76905003,186967,18202
7,deKH0pQ7-rg,2020-03-23T14:36:55Z,UCn6dNfiRE_Xunu7iMyvD7AA,Habertürk TV Canlı Yayın ᴴᴰ,Haberturk TV YouTube Kanalına Abone Ol ➤ http:...,Habertürk TV,76773593,56879,21747
8,CScQHGtL6qE,2020-04-08T14:00:04Z,UCUdbXlSI1MUFIk7DDzhJWKw,Five little monkeys 🙈 | 동요와 아이 노래 | 어린이 교육 | L...,Five little monkeys | 동요와 아이 노래 | 어린이 교육 | LaL...,LaLa kids Songs,73611070,206665,152949
9,zJj0f45FHOY,2020-03-24T17:59:52Z,UCszP2pSj7dKdFWmWMFS05jQ,"Stay Strong, Stay Healthy With Ensure&#39;s #1...","The healthier you are, the wealthier you can b...",Ensure India,70506634,19,7
